In [21]:
from skimage.measure import compare_ssim, compare_psnr
from scipy.misc import imread
import matplotlib.image as mpimg
import numpy as np
import os
from sklearn.metrics import mean_absolute_error
import cv2
import warnings
warnings.filterwarnings("ignore")




In [22]:
# This cell calculates the SSIM, PSNR, and MAE results according to the predicted image and target image.
files_path = '/data/jiayuan/SaveTestImage'

def max_ssim_function(a):
    item = [j for i,j in a]
    max_item = max(item)
    for i,j in a:
        if max_item==j:
            return i,j

def min_ssim_function(a):
    item = [j for i,j in a]
    max_item = min(item)
    for i,j in a:
        if max_item==j:
            return i,j

for files in os.listdir(files_path):
    for category in os.listdir(os.path.join(files_path,files)):
        for x2x in os.listdir(os.path.join(files_path,files,category)):
            max_ssim=[]
            max_psnr=[]
            min_mae=[]
            for epoch in os.listdir(os.path.join(files_path,files,category,x2x)):
                if epoch == '200':
                    ssim_amount = []
                    mae_amount = []
                    psnr_amount = []
                    with open('evaluate_10.txt','a') as f: 
                        write_or = os.path.join(files_path,files,category,x2x,epoch)
#                         f.write(write_or+'\n')
                        for image in os.listdir(os.path.join(files_path,files,category,x2x,epoch)):
                            img = mpimg.imread(os.path.join(files_path,files,category,x2x,epoch,image))
                            img = img[:,:,0]
                            img = img*2-1
#                             img = cv2.imread(os.path.join(files_path,files,category,x2x,epoch,image), cv2.IMREAD_GRAYSCALE)
                            img1 = img[:,256:512]
                            img2 = img[:,512:]
#                             mae_img1 = (img1+1)*127.5
#                             mae_img2 = (img2+1)*127.5
    #                         img2 = np.resize(img2, (img1.shape[0], img1.shape[1], img1.shape[2]))
                            mae = mean_absolute_error(img2, img1)
                            psnr = compare_psnr(img2,img1)
                            ssim = compare_ssim(img1, img2, multichannel=True)
                            ssim_amount.append(ssim)
                            mae_amount.append(mae)
                            psnr_amount.append(psnr)
                        average_ssim = sum(ssim_amount)/len(ssim_amount)
                        average_mae = sum(mae_amount)/len(mae_amount)
                        average_psnr = sum(psnr_amount)/len(psnr_amount)
                        max_ssim.append([epoch,average_ssim])
                        max_psnr.append([epoch,average_psnr])
                        min_mae.append([epoch,average_mae])
                        writes = files+'_'+category+'_'+x2x+': '+ str(average_mae)+','+ str(average_psnr)+','+ str(average_ssim)
                        f.write(writes+'\n')
                        print('File: {}. category:{}. {}. The eopch {} average_mae is {}, average_psnr is {}, average_ssim is {} '.format(files,category,x2x,epoch,str(average_mae),str(average_psnr),str(average_ssim)))
                
                    

File: commonG_rate1. category:datatime6. b2a. The eopch 200 average_mae is 0.23456376492977143, average_psnr is 11.921977011385534, average_ssim is 0.5521365447650135 
commonG_rate1_datatime6_b2a_('200', 0.5521365447650135)
commonG_rate1_datatime6_b2a_('200', 11.921977011385534)
commonG_rate1_datatime6_b2a_('200', 0.23456376492977143)
File: commonG_rate1. category:datatime6. a2b. The eopch 200 average_mae is 0.12880315110087395, average_psnr is 18.04076944547917, average_ssim is 0.6016822723343951 
commonG_rate1_datatime6_a2b_('200', 0.6016822723343951)
commonG_rate1_datatime6_a2b_('200', 18.04076944547917)
commonG_rate1_datatime6_a2b_('200', 0.12880315110087395)
File: commonG_rate1. category:datatime8. b2a. The eopch 200 average_mae is 0.22039071917533876, average_psnr is 12.555676547446867, average_ssim is 0.5342542450688057 
commonG_rate1_datatime8_b2a_('200', 0.5342542450688057)
commonG_rate1_datatime8_b2a_('200', 12.555676547446867)
commonG_rate1_datatime8_b2a_('200', 0.2203907191

ValueError: max() arg is an empty sequence

In [23]:
# This cell can extract the result to excel and calculate the standard deviation and average.
import xlwt 

import xlrd 

import re

import numpy as np
import pandas as pd

fopen=open("evaluate_10.txt",'r')

lines=fopen.readlines()

# create a excel file

file=xlwt.Workbook(encoding='utf-8',style_compression=0)

# create a sheet

sheet=file.add_sheet('data')

############################
i=0
sheet.write(i,0,'datasets')
sheet.write(i,1,'method')
sheet.write(i,2,'translate')
sheet.write(i,3,'average_mae')
sheet.write(i,4,'average_psnr')
sheet.write(i,5,'average_ssim')


i+=1


MAE = {'data_a2b':[], '368dataset_a2b':[], 'T1-T2_a2b':[],'selected368_a2b':[],'data_b2a':[], '368dataset_b2a':[], 'T1-T2_b2a':[],'selected368_b2a':[]}
PSNR = {'data_a2b':[], '368dataset_a2b':[], 'T1-T2_a2b':[],'selected368_a2b':[],'data_b2a':[], '368dataset_b2a':[], 'T1-T2_b2a':[],'selected368_b2a':[]}
SSIM = {'data_a2b':[], '368dataset_a2b':[], 'T1-T2_a2b':[],'selected368_a2b':[],'data_b2a':[], '368dataset_b2a':[], 'T1-T2_b2a':[],'selected368_b2a':[]}

for line in lines:
    if 'trip' or 'common' in line:
        print(line)
        a=line.split('_')
    #     print(a)
        dataset = a[2].split('time')[0]
        method = a[0]
        if 'b2a' in a[3]:
            translate = 'b2a'
        else:
            translate = 'a2b'
        mae = float(a[3].split(': ')[1].split(',')[0])
        psnr = float(a[3].split(': ')[1].split(',')[1])
        ssim = float(a[3].split(': ')[1].split(',')[2].split('\n')[0])
        MAE[dataset+'_'+translate].append(mae)
        PSNR[dataset+'_'+translate].append(psnr)
        SSIM[dataset+'_'+translate].append(ssim)


        sheet.write(i,0,dataset)
        sheet.write(i,1,method)
        sheet.write(i,2,translate)
        sheet.write(i,3,mae)
        sheet.write(i,4,psnr)
        sheet.write(i,5,ssim)
        i=i+1
        file.save('trip_common.xls')

        
# below code calculate standard deviation and average of MAE, SSIM and PSNR
# df = pd.DataFrame(data = MAE)
# Stand_MAE = df.std()
# Ave_MAE = df.mean()

# df = pd.DataFrame(data = PSNR)
# Stand_PSNR = df.std()
# Ave_PSNR = df.mean()

# df = pd.DataFrame(data = SSIM)
# Stand_SSIM = df.std()
# Ave_SSIM = df.mean()

# with pd.ExcelWriter('trip-ce-ssim.xls') as writer:
#     Ave_MAE.to_excel(writer,index=True,sheet_name='Ave_MAE')
#     Stand_MAE.to_excel(writer,index=True,sheet_name='Stand_MAE')
#     Ave_PSNR.to_excel(writer,index=True,sheet_name='Ave_PSNR')
#     Stand_PSNR.to_excel(writer,index=True,sheet_name='Stand_PSNR')
#     Ave_SSIM.to_excel(writer,index=True,sheet_name='Ave_SSIM')
#     Stand_SSIM.to_excel(writer,index=True,sheet_name='Stand_SSIM')




commonG_rate1_datatime6_b2a: 0.23456376492977143,11.921977011385534,0.5521365447650135

commonG_rate1_datatime6_a2b: 0.12880315110087395,18.04076944547917,0.6016822723343951

commonG_rate1_datatime8_b2a: 0.22039071917533876,12.555676547446867,0.5342542450688057

commonG_rate1_datatime8_a2b: 0.13344372138381005,17.80109242004368,0.5916408453687232

commonG_rate1_datatime4_b2a: 0.21133165210485458,12.746499045669902,0.5703340046430421

commonG_rate1_datatime4_a2b: 0.12606159821152688,18.277922984985242,0.6046348387711118

commonG_rate1_datatime9_b2a: 0.20049781054258348,13.461398790618622,0.5554118907021415

commonG_rate1_datatime9_a2b: 0.1290021948516369,18.124811878983895,0.6069380853320323

commonG_rate1_datatime1_b2a: 0.1721864566206932,14.171493513196769,0.6020053346747716

commonG_rate1_datatime1_a2b: 0.1007440060377121,20.56438750550719,0.6455148916679536

commonG_rate1_datatime_b2a: 0.21379834562540054,12.93610064500823,0.5458497577415282

commonG_rate1_datatime_a2b: 0.1304897315